# SUBSETTING HUMAN TRANSCRIPTOME 

We explore human transcriptome, coming from 
[APPRIS](http://appris-tools.org/)
principal isoforms, and do some sanity checks. Based on our observatioons, we make decisions about which transcripts to exclude.

Note that we are inside the scripts folder and need to setup our paths accordingly.

In [1]:
! ls ..

raw  riboflow_annot_and_ref  scripts


To start, we include our library folder first. Make sure it exists in two levels up

In [2]:
import sys
sys.path.insert(0, '../../../..')
import gzip

from collections import defaultdict

import ref_lib
from ref_lib.Fasta import FastaFile, FastaEntry
from ref_lib.GTF import GTFfile, GTFEntry, get_gtf_contents

Here are the reference files we will be using:

In [3]:
transcripts_fasta = "../raw/sequence/gencode.v29.transcripts.fa.gz"
gencode_gtf       = "../raw/annotation/gencode.v29.annotation.gtf.gz"
appris_file       = "../raw/annotation/appris_data.principal.txt.gz"

Let's check the documentation and see what the output of get_gtf_contents is like

In [4]:
help(get_gtf_contents)

Help on function get_gtf_contents in module ref_lib.GTF:

get_gtf_contents(gtf_file)
    Reads a gtf file into a dictionary. 
    
    NOTE 1:
    We tested this on encode gtf file.
    Some unexpected behavior might occur on GTF files from other sources.
    
    NOTE 2:
    For compatibility reasons, in gene_id and transcript_id, we are removing the part after
    ".". So, for example, ENSG00000178199.13 becomes ENSG00000178199
    
    It returns a dictionary where its keys are gene ids and values are again (sub)dictionaries
    where each (sub dictionary is of the form)
        transcript_id -> { "exons": list(), "CDS": list(), "strand": k.strand,
                           "start": k.start, "end": k.end, length: int}
                           
    Later when we want to determine relative UTR, CDS coordinates  etc. 
    We can place them in these (sub)dictionaries



GENCODE gtf contents are stored in a dictionary. It might take a while to read all gtf:

In [5]:
gtf_all = get_gtf_contents( gencode_gtf ) 

Random Two entries from the gtf contents:

In [6]:
list(gtf_all.items())[1000:1002]

[('ENSG00000162512',
  {'ENST00000336798': {'exons': [(30876552, 30878740),
     (30874297, 30874588),
     (30869467, 30873377)],
    'CDS': [(30876552, 30877247, 0),
     (30874297, 30874588, 1),
     (30873214, 30873377, 2)],
    'strand': '-',
    'start': 30869467,
    'end': 30878740,
    'gene_type': 'protein_coding',
    'length': 6392},
   'ENST00000339394': {'exons': [(30908449, 30908761),
     (30878623, 30878740),
     (30876552, 30877165),
     (30874297, 30874588),
     (30873135, 30873377)],
    'CDS': [(30908449, 30908586, 0),
     (30878623, 30878740, 1),
     (30876552, 30877165, 2),
     (30874297, 30874588, 3),
     (30873214, 30873377, 4)],
    'strand': '-',
    'start': 30873135,
    'end': 30908761,
    'gene_type': 'protein_coding',
    'length': 1580},
   'ENST00000471567': {'exons': [(30881358, 30881429),
     (30878623, 30878740),
     (30876767, 30877165)],
    'CDS': [],
    'strand': '-',
    'start': 30876767,
    'end': 30881429,
    'gene_type': 'prote

We define the functions to extract CDS and UTR coordinates relative ro the transcript start.
These care stored in the gtf_all dictionary with "bed_" prefix.

In [7]:
def _find_rel_cds_end_coords( exon_list, cds_contents, coord_type, strand ):
    """
    Returns the start or end of the CDS 0-based bed format
    So the start position is included and end position is excluded
    """    
    # c is the corresponding_exon_index to the cds chunk
    
    if coord_type == "start":
        cds_chunk = cds_contents[0]
        c = cds_chunk[2]
        if strand == "+":
            this_sum = cds_chunk[0] - exon_list[c][0]
        else:
            this_sum = exon_list[c][1] - cds_chunk[1]
    else:
        cds_chunk = cds_contents[-1]
        c = cds_chunk[2]
        this_sum = (cds_chunk[1] - cds_chunk[0]) + 1
        
        if len(cds_contents) == 1:
            if strand == "+":
                this_sum += cds_chunk[0]  - exon_list[c][0]  
            else:
                this_sum += exon_list[c][1] - cds_chunk[1]
        """
        Delete this later!!!
        if strand == "+":
            this_sum = (cds_chunk[1] - cds_chunk[0]) + 1
        else:
            this_sum = cds_chunk[0] - exon_list[c][0]
        """
    for e in exon_list[:c]:
        this_sum += (e[1] - e[0]) + 1
        
    return this_sum


def find_rel_utr_cds_positions( genes_dict ):
    """
    For each transcri,pt, the first nucleotide (of the transcript) has position 0
    the coordinates are in BED stye
    so For A,B 
    A: start, is INCLUDED
    B: end, is EXCLUDED
    The coordinates have the _bed prefix and they are stored in
    the corresponding transcript dictionary
    """
    # 
    # 0-based and end coordinate is excluded
    # 
    for g_name, transcripts in genes_dict.items():
    
        for t_name, t_contents in transcripts.items():
            CDS_contents = t_contents.get("CDS", list())
            if len(CDS_contents) == 0:
                continue

            cds_rel_start = \
                _find_rel_cds_end_coords( t_contents["exons"], CDS_contents, "start", t_contents["strand"]   )
            cds_rel_end = \
                _find_rel_cds_end_coords( t_contents["exons"], CDS_contents, "end", t_contents["strand"] )
        
            t_length = t_contents["length"]
                
            t_contents["bed_CDS"] = ( cds_rel_start, cds_rel_end )
            if cds_rel_start > 0:
                t_contents["bed_UTR_5"] = (0, cds_rel_start  )
                
            if cds_rel_end < t_length:
                t_contents["bed_UTR_3"] = (cds_rel_end, t_length )

In [8]:
find_rel_utr_cds_positions(gtf_all)

Now we can see that they are in the dictionary:

In [9]:
list(gtf_all.items())[1000:1002]

[('ENSG00000162512',
  {'ENST00000336798': {'exons': [(30876552, 30878740),
     (30874297, 30874588),
     (30869467, 30873377)],
    'CDS': [(30876552, 30877247, 0),
     (30874297, 30874588, 1),
     (30873214, 30873377, 2)],
    'strand': '-',
    'start': 30869467,
    'end': 30878740,
    'gene_type': 'protein_coding',
    'length': 6392,
    'bed_CDS': (1493, 2645),
    'bed_UTR_5': (0, 1493),
    'bed_UTR_3': (2645, 6392)},
   'ENST00000339394': {'exons': [(30908449, 30908761),
     (30878623, 30878740),
     (30876552, 30877165),
     (30874297, 30874588),
     (30873135, 30873377)],
    'CDS': [(30908449, 30908586, 0),
     (30878623, 30878740, 1),
     (30876552, 30877165, 2),
     (30874297, 30874588, 3),
     (30873214, 30873377, 4)],
    'strand': '-',
    'start': 30873135,
    'end': 30908761,
    'gene_type': 'protein_coding',
    'length': 1580,
    'bed_CDS': (175, 1501),
    'bed_UTR_5': (0, 175),
    'bed_UTR_3': (1501, 1580)},
   'ENST00000471567': {'exons': [(308

## Selecting Longest Appris Transcripts

When going through Appris transcripts, we pick only the "PRINCIPAL*" ones.

In [10]:
def read_principal_appris_trascript_list(appris_file, gtf_contents):
    """
    Reads the appris isoform file into a dictionary
    
    We assume that the files is of the form:
    Gene NAme \t GENE_ID \t TRANSCRIOPT_ID \t APPRIS_CATEGORY
    # C1orf112	ENSG00000000460	ENST00000286031	CCDS1285	PRINCIPAL:1
    
    The output dictionary is of the form:
    appris_genes[gene_id][transcript_id] = {  "gene_name": gene_name, "category": category }
    """
    
    appris_genes = defaultdict(dict)
    myopen=open
    if appris_file.endswith(".gz"):
        myopen = gzip.open
    
    with myopen(appris_file, "rt") as input_stream:
        for entry in input_stream:
            contents = entry.strip().split()
            if len(contents) < 5:
                continue
            gene_name, gene_id, transcript_id, dummy, category = contents
            if not category.startswith("PRINCIPAL"):
                continue
            appris_genes[gene_id][transcript_id] = gtf_contents[gene_id][transcript_id]
            
    return appris_genes

In [11]:
appris_principal_transcripts = read_principal_appris_trascript_list(appris_file, gtf_all)
len(appris_principal_transcripts)

20371

In [12]:
def pick_longest_appris_transcripts( appris_genes ):
    longest_picks = defaultdict(dict)
    
    for gene, transcripts in appris_genes.items():
        g_transcripts = list( transcripts.items() )
        longest_transcript = g_transcripts[0]
        for t_name, t_contents in g_transcripts[1:]:
            if t_contents["length"] > longest_transcript[1]["length"]:
                longest_transcript = (t_name, t_contents)
        longest_picks[ gene ][longest_transcript[0]] = longest_transcript[1]   
    return longest_picks

In [13]:
longest_appris_transcripts = pick_longest_appris_transcripts(appris_principal_transcripts)
len(longest_appris_transcripts)

20371

How many of these transcripts do NOT have CDS?

In [14]:
no_cds_transcripts = defaultdict(dict)
for g, transcripts in longest_appris_transcripts.items():
    # There is only one (longest) transcript so pick that
    t_name, t_contents = list(transcripts.items())[0]
    CDS = t_contents.get("bed_CDS", None)
    if CDS == None:
        no_cds_transcripts[g][t_name] = t_contents
print(len(no_cds_transcripts))    

0


Great! So all appris transcripts have CDS region annotated!

Just for verification, let's look at GAPDH and BRCA2 and see if we picked the longest transcript:

In [15]:
for t_name, t_contents in appris_principal_transcripts["ENSG00000111640"].items():
    print(t_name, t_contents["length"])
print("--------")
for t_name, t_contents in appris_principal_transcripts["ENSG00000139618"].items():
    print(t_name, t_contents["length"])

ENST00000396859 1256
ENST00000229239 1875
ENST00000396861 1348
--------
ENST00000544455 10984
ENST00000380152 11986


In [16]:
longest_appris_transcripts["ENSG00000111640"]

{'ENST00000229239': {'exons': [(6533927, 6534569),
   (6534810, 6534861),
   (6536494, 6536593),
   (6536684, 6536790),
   (6536920, 6537010),
   (6537101, 6537216),
   (6537309, 6537390),
   (6537584, 6537996),
   (6538101, 6538371)],
  'CDS': [(6534833, 6534861, 1),
   (6536494, 6536593, 2),
   (6536684, 6536790, 3),
   (6536920, 6537010, 4),
   (6537101, 6537216, 5),
   (6537309, 6537390, 6),
   (6537584, 6537996, 7),
   (6538101, 6538167, 8)],
  'strand': '+',
  'start': 6533927,
  'end': 6538371,
  'gene_type': 'protein_coding',
  'length': 1875,
  'bed_CDS': (666, 1671),
  'bed_UTR_5': (0, 666),
  'bed_UTR_3': (1671, 1875)}}

In [17]:
longest_appris_transcripts["ENSG00000139618"]

{'ENST00000380152': {'exons': [(32315474, 32315667),
   (32316422, 32316527),
   (32319077, 32319325),
   (32325076, 32325184),
   (32326101, 32326150),
   (32326242, 32326282),
   (32326499, 32326613),
   (32329443, 32329492),
   (32330919, 32331030),
   (32332272, 32333387),
   (32336265, 32341196),
   (32344558, 32344653),
   (32346827, 32346896),
   (32354861, 32355288),
   (32356428, 32356609),
   (32357742, 32357929),
   (32362523, 32362693),
   (32363179, 32363533),
   (32370402, 32370557),
   (32370956, 32371100),
   (32376670, 32376791),
   (32379317, 32379515),
   (32379750, 32379913),
   (32380007, 32380145),
   (32394689, 32394933),
   (32396898, 32397044),
   (32398162, 32400266)],
  'CDS': [(32316461, 32316527, 1),
   (32319077, 32319325, 2),
   (32325076, 32325184, 3),
   (32326101, 32326150, 4),
   (32326242, 32326282, 5),
   (32326499, 32326613, 6),
   (32329443, 32329492, 7),
   (32330919, 32331030, 8),
   (32332272, 32333387, 9),
   (32336265, 32341196, 10),
   (3234

This gives us confidence that we picked the longest principal transcripts

## Sanity Checks

At this point, we have the relative coordinates of CDS and UTRs, and we picked the longest principal appris transcripts.
Now, we can do some sanity checks to see if there are any issues with our implmentation.

These sanity checks are:
    1. The length of CDS should be divisible by 3
    2. The start codon should be ATG. The stop codon should be one of TAG, TGA, TAA
    
Also, it is claimed that the alternative isoforms for the principal Appris genes have identical CDS. We walso wan to confirm this.
    3. CDS regions

### 1. CDS Length

In [18]:
# The remainders are stored at the corresponding index position
# i.e., CDS_remainder_by_3[1] will hold the total number of remainders whose CDS length is of the form 3m + 1
CDS_remainder_by_3 = [0,0,0]
non_zero_remainder_transcripts = defaultdict(dict)

for g, transcripts in longest_appris_transcripts.items():
    for t_name, t_contents in transcripts.items():
        bed_CDS = t_contents.get("bed_CDS", None)
        this_remainder = (bed_CDS[1] - bed_CDS[0]) % 3
        CDS_remainder_by_3[this_remainder] += 1
        if this_remainder != 0:
            non_zero_remainder_transcripts[g][t_name] = t_contents

print("CDS length divisible by 3: ", CDS_remainder_by_3[0])
print("CDS length NOT divisible by 3: ", CDS_remainder_by_3[1] + CDS_remainder_by_3[2])


CDS length divisible by 3:  19959
CDS length NOT divisible by 3:  412


Most of the transcripts have CDS multiple of 3. But some of them don't. Now we look at what kind of transcripts they are:

In [19]:
CDS_non_zero_remainder_plus_strand = 0
one_exon_non_zero_remainder_cases = list()
gene_types = defaultdict(int)

for g, transcripts in non_zero_remainder_transcripts.items():
     for t_name, t_contents in transcripts.items():
        gene_types[gtf_all[g][t_name]["gene_type"]] +=1  
        if t_contents["strand"] == "+":
            CDS_non_zero_remainder_plus_strand += 1
        if len(t_contents["CDS"]) == 1:
            one_exon_non_zero_remainder_cases.append( t_name)

all_total = 0
for g_type, g_count in gene_types.items():
    print(g_type, g_count)
    all_total += g_count
print(all_total)

polymorphic_pseudogene 16
protein_coding 148
IG_C_gene 7
IG_J_gene 15
IG_V_gene 67
TR_J_gene 31
TR_C_gene 6
TR_V_gene 89
IG_D_gene 31
TR_D_gene 2
412


In [20]:
all_gene_types = defaultdict(int)
total_t = 0

for g, transcripts in appris_principal_transcripts.items():
    for t_name, t_contents in transcripts.items():
        all_gene_types[ t_contents["gene_type"] ] +=1
        total_t += 1
        
        
for  g_type, g_count in all_gene_types.items():
    print(g_type, g_count)
print(total_t)

protein_coding 26784
polymorphic_pseudogene 42
IG_C_gene 14
IG_J_gene 18
IG_V_gene 144
TR_J_gene 79
TR_C_gene 6
TR_V_gene 106
TR_D_gene 4
IG_D_gene 37
27234


In [21]:
print(len(one_exon_non_zero_remainder_cases), "\n" ,  one_exon_non_zero_remainder_cases[:20])

118 
 ['ENST00000641057', 'ENST00000504970', 'ENST00000377712', 'ENST00000390237', 'ENST00000390238', 'ENST00000390239', 'ENST00000390240', 'ENST00000390242', 'ENST00000620613', 'ENST00000429239', 'ENST00000394191', 'ENST00000506619', 'ENST00000514827', 'ENST00000587816', 'ENST00000439431', 'ENST00000390333', 'ENST00000390337', 'ENST00000390338', 'ENST00000390341', 'ENST00000390412']


A closer look at the transcript ENST00000426406 directly on the file: gencode.v29.annotation.gtf.gz

In [22]:
! zcat ../annotation/gencode.v29.annotation.gtf.gz | grep ENST00000377712 

gzip: ../annotation/gencode.v29.annotation.gtf.gz: No such file or directory


This pseodogene looks problematic in the reference!

In [23]:
longest_appris_transcripts["ENSG00000204872"]["ENST00000377712"]

{'exons': [(73700576, 73701340)],
 'CDS': [(73700627, 73701305, 0)],
 'strand': '-',
 'start': 73700576,
 'end': 73701340,
 'gene_type': 'polymorphic_pseudogene',
 'length': 765,
 'bed_CDS': (35, 714),
 'bed_UTR_5': (0, 35),
 'bed_UTR_3': (714, 765)}

In [24]:
73701305 - 73700627 +1

679

In [25]:
714 - 35

679

Both our calculations and eye-inspection of the GTF file tell us that CDS length of this gene is 679 and is not divisible by 3

### 2. Start & Stop Codons

In [26]:
t_fasta_stream = FastaFile(transcripts_fasta)

clipped = list()
start_triplets = defaultdict(int)
stop_triplets = defaultdict(int)

for this_fasta_entry in t_fasta_stream:
    contents = this_fasta_entry.header.strip().split("|")
    this_t = contents[0].split(".")[0]
    
    g_contents = contents[1].split(".")
    this_g = g_contents[0]
    if len(g_contents) >= 2 and "PAR_Y" in g_contents[1]:
        continue
    
    transcripts = longest_appris_transcripts.get(this_g, None)
    if bed_CDS == None:
        continue
    if this_t in list(longest_appris_transcripts[this_g].keys()):
        bed_CDS = longest_appris_transcripts[this_g][this_t].get("bed_CDS", None)
        
        if bed_CDS != None:
            start_codon_coord = bed_CDS[0]
            start_triplets[ this_fasta_entry.sequence[ start_codon_coord:start_codon_coord+3 ]  ] += 1
            stop_triplets[ this_fasta_entry.sequence[ bed_CDS[1]:bed_CDS[1] + 3 ]  ] += 1

print(start_triplets)
print("---------------------------")
print(stop_triplets)

defaultdict(<class 'int'>, {'ATG': 19995, 'GTG': 17, 'AGG': 8, 'AAG': 9, 'GCT': 7, 'GAA': 11, 'TGA': 23, 'CTG': 23, 'GGT': 10, 'GGG': 15, 'CTC': 10, 'GGA': 12, 'CTT': 6, 'AGT': 3, 'GAT': 10, 'ATT': 9, 'TGT': 3, 'GAC': 3, 'GAG': 6, 'GCG': 6, 'CAG': 7, 'AGC': 8, 'CCT': 7, 'TTC': 3, 'GCC': 8, 'ACT': 8, 'TTG': 4, 'ATC': 4, 'ATA': 11, 'TCC': 7, 'AAC': 4, 'CAC': 3, 'ACG': 2, 'TAA': 6, 'AGA': 9, 'CTA': 4, 'CAA': 6, 'TAG': 1, 'CGA': 1, 'ACC': 2, 'GTA': 11, 'TCT': 3, 'TTA': 3, 'CGC': 4, 'CGG': 1, 'TAT': 3, 'GCA': 3, 'TTT': 3, 'GTC': 5, 'AAA': 2, 'ACA': 5, 'CCA': 4, 'TGG': 7, 'TCA': 6, 'CAT': 1, 'TAC': 5, 'GTT': 3, 'CCG': 3, 'TGC': 2, 'GGC': 2, 'CGT': 1, 'CCC': 2, 'AAT': 1})
---------------------------
defaultdict(<class 'int'>, {'TAG': 4433, 'TAA': 5640, 'TGA': 9798, '': 493, 'ACA': 1, 'TAT': 1, 'GTG': 1, 'ATA': 1, 'TGG': 1, 'GAC': 1, 'TTG': 1})


In [27]:
lenght_dict = defaultdict(int)
short_and_p_coding = defaultdict(dict)
for g, transcripts in longest_appris_transcripts.items():
    for t_name , t_contents in transcripts.items():
        cds_length = t_contents["bed_CDS"][1] - t_contents["bed_CDS"][0]
        lenght_dict[ cds_length ] += 1
        if cds_length < 150 and t_contents["gene_type"] == "protein_coding":
            short_and_p_coding[g][t_name] = t_contents
            
print("There are", len(short_and_p_coding), "many short and protein coding transcripts (CDS length < 150)")

There are 77 many short and protein coding transcripts (CDS length < 150)


We think that these few short transcripts will not impact our results significantly. Also, very few reads are going to map to them.

### Selecting Only Protein-Coding Genes

Our sanity checks above suggest that many (though not all) of the transcripts that are failing CDS length and start / stop codon checks are not non-protein coding genes. So we decided to pick only protein-coding genes.

In [28]:
protein_coding_longest_transcripts = defaultdict(dict)
for g, transcripts in longest_appris_transcripts.items():
    for t_name , t_contents in transcripts.items():
        if t_contents["gene_type"] == "protein_coding":
            protein_coding_longest_transcripts[g][t_name] = t_contents
long_p_coding_count = len(protein_coding_longest_transcripts)
print("There are {} protein coding transcripts picked".format(long_p_coding_count))

There are 19922 protein coding transcripts picked


Now we revisit our sanity checks using the protein coding transcripts only

### 1. CDS Length (Revisited)

In [29]:
CDS_remainder_by_3 = [0,0,0]
non_zero_remainder_transcripts = defaultdict(dict)

for g, transcripts in protein_coding_longest_transcripts.items():
    for t_name, t_contents in transcripts.items():
        bed_CDS = t_contents.get("bed_CDS", None)
        this_remainder = (bed_CDS[1] - bed_CDS[0]) % 3
        CDS_remainder_by_3[this_remainder] += 1
        if this_remainder != 0:
            non_zero_remainder_transcripts[g][t_name] = t_contents

print("CDS length divisible by 3: ", CDS_remainder_by_3[0])
print("CDS length NOT divisible by 3: ", CDS_remainder_by_3[1] + CDS_remainder_by_3[2])

CDS length divisible by 3:  19774
CDS length NOT divisible by 3:  148


### 2. Start & Stop Codons (Revisited)

In [30]:
t_fasta_stream = FastaFile(transcripts_fasta)

clipped = list()
start_triplets = defaultdict(int)
stop_triplets = defaultdict(int)
empty_stop_triplets = defaultdict(dict)
transcripts_with_stop_codon = defaultdict(dict)

for this_fasta_entry in t_fasta_stream:
    contents = this_fasta_entry.header.strip().split("|")
    this_t = contents[0].split(".")[0]
    
    g_contents = contents[1].split(".")
    this_g = g_contents[0]
    if len(g_contents) >= 2 and "PAR_Y" in g_contents[1]:
        continue
    
    transcripts = protein_coding_longest_transcripts.get(this_g, None)

    if this_t in list(protein_coding_longest_transcripts[this_g].keys()):
        bed_CDS = protein_coding_longest_transcripts[this_g][this_t].get("bed_CDS", None)
        
        if bed_CDS != None:
            start_codon_coord = bed_CDS[0]
            start_triplets[ this_fasta_entry.sequence[ start_codon_coord:start_codon_coord+3 ]  ] += 1
            observed_stop_triplet = this_fasta_entry.sequence[ bed_CDS[1]:bed_CDS[1] + 3 ]
            stop_triplets[ observed_stop_triplet ] += 1
            if observed_stop_triplet == '':
                empty_stop_triplets[this_g][this_t] = protein_coding_longest_transcripts[this_g][this_t]
            else:
                transcripts_with_stop_codon[this_g][this_t] = protein_coding_longest_transcripts[this_g][this_t]

print("start_triplets")                
print(start_triplets)
print("---------------------------")
print("stop_triplets")  
print(stop_triplets)
print("---------------------------")
print("Number of transcripts_with_stop_codon = ", len(transcripts_with_stop_codon) )

start_triplets
defaultdict(<class 'int'>, {'ATG': 19713, 'GTG': 11, 'AGG': 3, 'AAG': 9, 'GCT': 4, 'GAA': 4, 'TGA': 4, 'CTG': 22, 'GGT': 1, 'GGG': 7, 'CTC': 3, 'GGA': 6, 'CTT': 3, 'AGT': 3, 'GCG': 6, 'CAG': 5, 'AGC': 5, 'CCT': 6, 'TTC': 3, 'GCC': 5, 'ACT': 6, 'GAG': 2, 'TTG': 4, 'ATC': 4, 'ATA': 4, 'ATT': 7, 'TCC': 7, 'AAC': 3, 'CAC': 2, 'ACG': 2, 'TAA': 1, 'AGA': 5, 'GTA': 1, 'TCT': 2, 'CTA': 1, 'TTA': 2, 'ACC': 1, 'CGC': 4, 'CAA': 5, 'CGG': 1, 'TAT': 2, 'GAT': 4, 'GCA': 1, 'TTT': 1, 'GTC': 1, 'AAA': 1, 'GTT': 2, 'TCA': 4, 'TGG': 3, 'CCA': 3, 'GGC': 2, 'ACA': 3, 'CCG': 2, 'GAC': 1, 'TAC': 1, 'CGT': 1, 'TGC': 1, 'CCC': 1, 'AAT': 1})
---------------------------
stop_triplets
defaultdict(<class 'int'>, {'TAG': 4415, 'TAA': 5625, 'TGA': 9782, '': 100})
---------------------------
Number of transcripts_with_stop_codon =  19822


Clearly, picking only protein coding genes largely improved our results!

The empty triplets are coming from transcripts without 3UTR.

We can clearly see this from the GTF annotation or from our dictionary:

In [31]:
empty_stop_triplets

defaultdict(dict,
            {'ENSG00000204060': {'ENST00000641094': {'exons': [(41361931,
                 41362344),
                (41381616, 41382200),
                (41382202, 41382678)],
               'CDS': [(41361931, 41362344, 0),
                (41381616, 41382200, 1),
                (41382202, 41382678, 2)],
               'strand': '+',
               'start': 41361931,
               'end': 41382678,
               'gene_type': 'protein_coding',
               'length': 1476,
               'bed_CDS': (0, 1476)}},
             'ENSG00000274068': {'ENST00000357393': {'exons': [(108963361,
                 108963484),
                (108849537, 108849576),
                (108848912, 108849060),
                (108848752, 108848814),
                (108843134, 108843267),
                (108837660, 108837706)],
               'CDS': [(108963361, 108963474, 0),
                (108849537, 108849576, 1),
                (108848912, 108849060, 2),
                (10

When we examined these genes on UCSC browser, we saw they were overlapping with other genes and they didn't have a 3UTR.

We decided to take them out as well. 

Now we narrowed down our set to 
`transcripts_with_stop_codon`

In [32]:
t_fasta_stream = FastaFile(transcripts_fasta)

start_triplets = defaultdict(int)
stop_triplets = defaultdict(int)

for this_fasta_entry in t_fasta_stream:
    contents = this_fasta_entry.header.strip().split("|")
    this_t = contents[0].split(".")[0]
    
    g_contents = contents[1].split(".")
    this_g = g_contents[0]
    if len(g_contents) >= 2 and "PAR_Y" in g_contents[1]:
        continue
    
    transcripts = transcripts_with_stop_codon.get(this_g, None)
    if transcripts == None:
        continue
    if bed_CDS == None:
        continue
    if this_t in list(transcripts_with_stop_codon[this_g].keys()):
        bed_CDS = transcripts_with_stop_codon[this_g][this_t].get("bed_CDS", None)
        
        if bed_CDS != None:
            start_codon_coord = bed_CDS[0]
            start_triplets[ this_fasta_entry.sequence[ start_codon_coord:start_codon_coord+3 ]  ] += 1
            observed_stop_triplet = this_fasta_entry.sequence[ bed_CDS[1]:bed_CDS[1] + 3 ]
            stop_triplets[ observed_stop_triplet ] += 1
            
print(start_triplets)
print("---------------------------")
print(stop_triplets)

defaultdict(<class 'int'>, {'ATG': 19647, 'GTG': 7, 'AGG': 3, 'AAG': 9, 'GCT': 4, 'GAA': 4, 'CTG': 22, 'GGT': 1, 'GGG': 6, 'CTC': 3, 'GGA': 5, 'CTT': 3, 'AGT': 3, 'GCG': 4, 'CAG': 4, 'AGC': 5, 'TTC': 1, 'GCC': 4, 'ACT': 2, 'TTG': 3, 'ATC': 4, 'TGA': 3, 'ATA': 2, 'ATT': 6, 'TCC': 7, 'CAC': 2, 'ACG': 2, 'TAA': 1, 'AGA': 5, 'GTA': 1, 'TCT': 2, 'GAG': 1, 'CTA': 1, 'ACC': 1, 'CCT': 4, 'CAA': 3, 'CGG': 1, 'GAT': 4, 'AAC': 2, 'TAT': 1, 'TTA': 1, 'TTT': 1, 'GTC': 1, 'AAA': 1, 'GTT': 2, 'TCA': 3, 'TGG': 3, 'CCA': 3, 'CGC': 3, 'GGC': 2, 'ACA': 3, 'CCG': 2, 'GAC': 1, 'TAC': 1, 'CGT': 1, 'AAT': 1})
---------------------------
defaultdict(<class 'int'>, {'TAG': 4415, 'TAA': 5625, 'TGA': 9782})


In [33]:
len(transcripts_with_stop_codon)

19822

In [34]:
CDS_remainder_by_3 = [0,0,0]
non_zero_remainder_transcripts = defaultdict(dict)

for g, transcripts in transcripts_with_stop_codon.items():
    for t_name, t_contents in transcripts.items():
        bed_CDS = t_contents.get("bed_CDS", None)
        this_remainder = (bed_CDS[1] - bed_CDS[0]) % 3
        CDS_remainder_by_3[this_remainder] += 1
        if this_remainder != 0:
            non_zero_remainder_transcripts[g][t_name] = t_contents

print("CDS length divisible by 3: ", CDS_remainder_by_3[0])
print("CDS length NOT divisible by 3: ", CDS_remainder_by_3[1] + CDS_remainder_by_3[2])

CDS length divisible by 3:  19734
CDS length NOT divisible by 3:  88


### 3. CDS Regions

We check whether different transcripts of the same gene of the Appris list have different CDS sequence or not.

In [35]:
t_fasta_stream = FastaFile(transcripts_fasta)
selected_CDS_sequences = defaultdict(dict)
different_CDS_transcripts = list()

for this_fasta_entry in t_fasta_stream:
    contents = this_fasta_entry.header.strip().split("|")
    this_t = contents[0].split(".")[0]
    
    g_contents = contents[1].split(".")
    this_g = g_contents[0]
    if len(g_contents) >= 2 and "PAR_Y" in g_contents[1]:
        continue
    
    pre_transcripts = transcripts_with_stop_codon.get(this_g, None)
    if pre_transcripts == None or pre_transcripts == {}:
        continue
    
    transcripts = appris_principal_transcripts.get(this_g, {})
    if this_t not in transcripts.keys():
        continue
    cds_boundaries = transcripts[this_t]["bed_CDS"]
    
    selected_CDS_sequences[this_g][this_t] = this_fasta_entry.sequence[ cds_boundaries[0]: cds_boundaries[1] ]
                 
for g, transcripts in selected_CDS_sequences.items():
    
    transcript_items = list(transcripts.items ())
    first_t, first_t_cds = transcript_items[0] 
    
    for t_name, t_cds in transcript_items[1:]:
        if t_cds != first_t_cds:
            different_CDS_transcripts.append( (first_t, t_name, first_t_cds, t_cds) )

In [36]:
for e in different_CDS_transcripts:
    print( e[0], e[1], len(e[2]), len(e[3]) )

ENST00000619410 ENST00000621965 5046 5046
ENST00000612898 ENST00000396980 378 378
ENST00000369393 ENST00000629399 16788 16788
ENST00000242257 ENST00000440306 738 738
ENST00000369701 ENST00000369699 3666 3666
ENST00000358056 ENST00000616146 426 426
ENST00000393066 ENST00000262294 2892 2892
ENST00000328041 ENST00000613834 1932 1932


Interestingly, all different cds sequences have the same length. When we looked at the second item (ENST00000612898 ENST00000396980 378 378), the seuqneces looked almost identical. So we look at them nuc-by-nuc.

So, these two sequences at only one place (index 377).

We look at the longest sequence as well

In [37]:
def find_disagreeing_indices( seq_1, seq_2 ):
    disagreeing_indices = []
    
    for i in range( len(seq_1) ):
        if seq_1[i] != seq_2[i]:
            disagreeing_indices.append(i) 
    
    return disagreeing_indices

In [38]:
for e in different_CDS_transcripts:
    print( find_disagreeing_indices( e[2], e[3] ) )

[353]
[377]
[9878]
[5]
[621, 622, 624, 625, 626]
[200]
[2891]
[269]


Hence, appris principal transcripts have identical CDS. There are very few exceptions to this, coming from the above output. But, for mapping purposes this won't result in an important difference.

### 4. CDS Length Distirbution

In [39]:
cds_lengths = []
gene_lengths = []

for g, transcripts in transcripts_with_stop_codon.items():
    for t_name, t_contents in transcripts.items():
        cds_lengths.append( t_contents["bed_CDS"][1] - t_contents["bed_CDS"][0] )
        gene_lengths.append(t_contents["length"])
    
len(cds_lengths)

19822

In [40]:
import numpy as np

In [41]:
cds_lengths_np = np.array(cds_lengths, dtype= np.int)
gene_lengths_np = np.array(gene_lengths, dtype= np.int)

In [42]:
np.mean(cds_lengths)

1696.91615376854

In [43]:
np.median(cds_lengths)

1269.0

According to Karlin et. al.
[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1150220/](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1150220/),
the median size of a human protein is 375 peptides = 1125 nucleotides

This is close to (1269) what we got for our selected transcripts. 






## Summary & Conclusion

We explored [Appris](https://academic.oup.com/nar/article/46/D1/D213/4561658) transcripts. Based on our observations, we decided to filter out some transcripts for our purposes. In particular, we followed the following steps:

1. We picked only "PRINCIPAL" entries and left "ALTERNATIVE" entries.
2. We picked only protein coding genes by choosing "gene_type protein_coding" in the GTF file
3. We excluded transcripts that didn't have 3TR and hence a stop codon.

These steps are mostly based on our 'Sanity Checks'.

Also, we discard Y chromosome paralogs of genes. They have "PAR_Y" in their gene ID in the gtf file and the fasta file

After filtering, we had a total of 19822 transcripts.

* 19734 of these transcripts have CDS with length a multiple of 3. 88 of them don't.
* 19647 of them start with ATG (the start codon)
* All of the transcripts have proper stop codons
* All appris genes have identical CDS regions. However there are 8 exceptions to this. We found 8 cases where the transcripts have the same CDS length but differ in a few places. You can find the complete list above.
* The median length of the CDS regiosn is 1269. This is consistent with the median length of human proteins.



## Files and Hash Sums

Appris Version 2018_12.v28

Appris list was downloaded from : http://appris-tools.org/#/downloads
on January 17 2019

GENCODE:

All other files have been downloaded from gencode website: [https://www.gencodegenes.org/human/](https://www.gencodegenes.org/human/)

Version: Release 29 (GRCh38.p12)

In [44]:
! zcat  ../raw/sequence/gencode.v29.transcripts.fa.gz | md5sum

69765772b5a30104dd3874d82919be9f  -


In [45]:
! zcat ../raw/annotation/gencode.v29.annotation.gtf.gz | md5sum

737c63114c6d9cf1678a863e9087d179  -


In [46]:
! zcat ../raw/annotation/appris_data.principal.txt.gz | md5sum

28280c63e87392a3e1732c11cb4e3604  -


## Generating RiboFlow Reference and Annotation

If the riboflow reference and annotation has not already been generated, you can run the 'generate_riboflow_ref_and_annot.sh' inside the scripts folder.